In [1]:
import os
import sys

def get_project_path():
    import os
    parts = os.getcwd().rsplit("machine-learning", 1)
    return parts[0]
sys.path.append(get_project_path())

from data import DataManager

In [2]:
dm = DataManager("~/dataset")